In [ ]:
%tensorflow_version 1.x

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [ ]:
dataset = pd.read_csv("pulsar_stars.csv")

In [ ]:
def load_dataset():
    dataset = pd.read_csv("pulsar_stars.csv")
    feature = dataset[[ "Mean of the integrated profile",
                        "Standard deviation of the integrated profile",
                        "Excess kurtosis of the integrated profile", 
                        "Skewness of the integrated profile",
                        "Mean of the DM-SNR curve",
                        "Standard deviation of the DM-SNR curve",
                        "Excess kurtosis of the DM-SNR curve",
                        "Skewness of the DM-SNR curve"]]
    target = dataset[["target_class"]]

    normalizer = StandardScaler()
    feature = normalizer.fit_transform(feature)
    OHencoder = OneHotEncoder(sparse=False)
    target = OHencoder.fit_transform(target)

    return feature, target

feature, target = load_dataset()

In [ ]:
layer = {
    "input" : 8,
    "hidden" : 4,
    "output" : 2
}

weight = {
    "input_hidden" : tf.Variable(tf.random_normal(
        [layer["input"],layer["hidden"]]
    )),
    "hidden_output" : tf.Variable(tf.random_normal(
        [layer["hidden"], layer["output"]]
    ))
}

bias = {
    "input_hidden" : tf.Variable(tf.random_normal(
        [layer["hidden"]]
    )),
    "hidden_output" : tf.Variable(tf.random_normal(
        [layer["output"]]
    ))
}

In [ ]:
input_dataset = tf.placeholder(tf.float32, [None, layer["input"]])
output_dataset = tf.placeholder(tf.float32,[None, layer["output"]])

In [ ]:
def feed_forward(dataset):
    u1 = tf.matmul(dataset,weight["input_hidden"]) + bias["input_hidden"]
    y1 = tf.nn.sigmoid(u1)

    u2 = tf.matmul(y1,weight["hidden_output"]) + bias["hidden_output"]
    y2 = tf.nn.sigmoid(u2)

    return y2

output = feed_forward(input_dataset)

In [ ]:
epoch = 5000
alpha = 0.9123

In [ ]:
#calculate Loss
loss = tf.reduce_mean(0.5*(output_dataset - output)**2) 

In [ ]:
#optimize loss
optimizer = tf.train.GradientDescentOptimizer(alpha)
train = optimizer.minimize(loss)
#accuracy
accuracy = tf.equal(tf.argmax(output_dataset,axis=1,tf.argmax(output, axis=1))
accuracy = tf.reduce_mean(tf.cast(accuracy,tf.float32))

In [ ]:
feature_train, feature_test, target_train, target_test = train_test_split(feature, target, test_size=0.2, random_state=1)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    feed_train = {
          input_dataset: feature_train,
          output_dataset: target_train
    }

    feed_test = {
        input_dataset: feature_test,
        output_dataset: target_test
    }   

    #train
    for i in range(epoch):
        sess.run(train, feed_dict=feed_train)
        error = sess.run(loss, feed_dict=feed_train)

        if (i+1)%200 == 0:
            print("Epoch: {}, Error: {}".format(i+1,error))

    #testing
    print("Accuracy: {}\n".format(sess.run(accuracy*100, feed_dict = feed_test)))

Epoch: 200, Error: 0.028996704146265984
Epoch: 400, Error: 0.019676854833960533
Epoch: 600, Error: 0.016172293573617935
Epoch: 800, Error: 0.014378368854522705
Epoch: 1000, Error: 0.013285813853144646
Epoch: 1200, Error: 0.012545849196612835
Epoch: 1400, Error: 0.012029727920889854
Epoch: 1600, Error: 0.011636667884886265
Epoch: 1800, Error: 0.011329754255712032
Epoch: 2000, Error: 0.011094887740910053
Epoch: 2200, Error: 0.01091318391263485
Epoch: 2400, Error: 0.010769831947982311
Epoch: 2600, Error: 0.010653979144990444
Epoch: 2800, Error: 0.010557967238128185
Epoch: 3000, Error: 0.01047660131007433
Epoch: 3200, Error: 0.010406364686787128
Epoch: 3400, Error: 0.010344810783863068
Epoch: 3600, Error: 0.01029018871486187
Epoch: 3800, Error: 0.010241221636533737
Epoch: 4000, Error: 0.010196929797530174
Epoch: 4200, Error: 0.010156575590372086
Epoch: 4400, Error: 0.010119556449353695
Epoch: 4600, Error: 0.010085408575832844
Epoch: 4800, Error: 0.010053747333586216
Epoch: 5000, Error: 0.0